逻辑斯蒂回归(LogisticRegression)是一个重要的分类模型，该算法将线性回归的输出概率化，根据“概率”进行分类

In [1]:
import math
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pylab import scatter, show, legend, xlabel, ylabel
import matplotlib.pyplot as plt
import os

# 数据：学科成绩为特征的二分类问题

In [2]:
df = pd.read_csv("data.csv", header=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   grade1     100 non-null    float64
 1   grade2     100 non-null    float64
 2   label;;;;  100 non-null    object 
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [3]:
df.columns = ["grade1","grade2","label"] 
#df的第三个列名有；；；
x = df["label"].map(lambda x: float(x.rstrip(';')))
X = df[["grade1","grade2"]]
X = np.array(X)
print(X)
#数据min_max归一化到（-1，1）间 
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
X = min_max_scaler.fit_transform(X)
#数据集label中有；；；；要去掉
Y = df["label"].map(lambda x: float(x.rstrip(';')))
Y = np.array(Y)
print(Y)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.33)

[[34.62365962 78.02469282]
 [30.28671077 43.89499752]
 [35.84740877 72.90219803]
 [60.18259939 86.3085521 ]
 [79.03273605 75.34437644]
 [45.08327748 56.31637178]
 [61.10666454 96.51142588]
 [75.02474557 46.55401354]
 [76.0987867  87.42056972]
 [84.43281996 43.53339331]
 [95.86155507 38.22527806]
 [75.01365839 30.60326323]
 [82.30705337 76.4819633 ]
 [69.36458876 97.71869196]
 [39.53833914 76.03681085]
 [53.97105215 89.20735014]
 [69.07014406 52.74046973]
 [67.94685548 46.67857411]
 [70.66150955 92.92713789]
 [76.97878373 47.57596365]
 [67.37202755 42.83843832]
 [89.67677575 65.79936593]
 [50.53478829 48.85581153]
 [34.21206098 44.2095286 ]
 [77.92409145 68.97235999]
 [62.27101367 69.95445795]
 [80.19018075 44.82162893]
 [93.1143888  38.80067034]
 [61.83020602 50.25610789]
 [38.7858038  64.99568096]
 [61.37928945 72.80788731]
 [85.40451939 57.05198398]
 [52.10797973 63.12762377]
 [52.04540477 69.43286012]
 [40.23689374 71.16774802]
 [54.63510555 52.21388588]
 [33.91550011 98.86943574]
 

# 设定拟合函数（Hypothesis Function）

逻辑斯蒂回归将线性回归拟合函数的输出通过Sigmoid函数映射到概率区间[0,1]，利用概率构造二分类模型（实际上是分类模型）

线性回归函数： $z=\theta^Tx$

Sigmoid函数： $g(z) = \frac{1}{1+e^{-z}}$

<img src="./sigmoid.png" width=500 height=5000>

逻辑斯蒂拟合函数(Hypothesis Function)：$h_\theta(x) = \frac{1}{1+e^{-\theta^Tx}}$

In [4]:
#构造逻辑斯蒂拟合函数
def Sigmoid(z):
    """
    Sigmoid函数：输入为线性回归函数值
    """
    sig_z = 1 / (1+ np.exp(-z))
    return sig_z 
def Hypothesis(theta, x):
    """
    逻辑斯蒂拟合函数：线性回归函数和Sigmoid函数的复合
    输入：theta(线性回归学习权重，与数据的维度相同)、x(数据)
    输出：Sigmoid(z)（拟合函数）(z为线性回归函数)
    """
    z = np.dot(theta, x)
    return Sigmoid(z)


# 设定代价函数（Cost Function）

线性回归的代价函数（均方误差）对于sigmoid()是非凸的,容易使优化落入局部极小值。
因此借鉴概率论中极大似然估计的思想，选用以下代价函数，可保证优化的效果：

<img src="./costfun1.png" width=500 height=5000>

该式可写为：

<img src="./costfun2.png" width=500 height=5000>

可以看出，这样的代价函数在y=1的前提下，如果预测值越接近1，那么相应代价就越小，反之则越大（对于y=0亦然）

设m为样本数：

<img src="./costfun3.png" width=1000 height=1000>

In [5]:
def Cost_Function(X,Y,theta):
    """
    代价函数：按标签对每个样本的“损失”进行累加并平均
    输入：X（数据）、Y（标签）、theta(当前theta)
    输出：J（损失函数值）
    """
    J = 0
    for i in (0,len(Y)-1):
        z = Hypothesis(theta, X[i])
        Z = Sigmoid(z) # 这个相当于是h_theta(x)
        if Y[i] == 1:
            J += -np.log(Z)
        if Y[i] == 0:
            J += -np.log(1 - Z)
        J = J / len(X)
    return J

# 对代价函数进行梯度下降优化Theta

关于Theta中的第j个权重$\theta_j$,其一轮的梯度下降更新为：$\theta_j^{'} = \theta_j-\frac{\alpha}{m} \sum_{i=1}^m(h_\theta(x^{(i)}-y^{(i)})x_j^{(i)}$

In [6]:
def Gradient_Descent(X,Y,theta,alpha):
    """
    一轮梯度下降更新theta
    输入：X（数据）、Y（标签）、theta(当前theta，要被更新)、alpha（学习率）
    输出：new_theta（更新的theta）
    """
    m = len(Y)
    new_theta = theta
    for j in range(len(theta)):
        sumErrors = 0
        for i in range(m):
            xi = X[i]
            yi = Y[i]
            h = Hypothesis(theta,X[i])
            sumErrors += (h-yi)*xi[j]
        new_theta[j] = theta[j] - alpha*1.0/m*sumErrors
    
#     m = len(Y)
#     num_features = len(X[0])  # Number of features
#     new_theta = np.copy(theta)
#     for j in range(num_features):
#         gradient = 0
#         for i in range(m):
#             # Calculate the gradient for each feature
#             z = np.dot(theta, X[i])
#             # print(z)
#             h = Sigmoid(z)
#             # print(h)
#             gradient += (h - Y[i]) * X[i][j]
#         # print(gradient)
        
#         # Update the j-th weight in theta
#         # print(alpha * (1 / m) * gradient)
#         new_theta[j] = new_theta[j] - alpha * (1 / m) * gradient
    return new_theta

# 逻辑斯蒂回归

In [7]:
def Logistic_Regression(X,Y,alpha,theta,num_iters):
    """
    多轮迭代梯度下降
    输入：X（数据）、Y（标签）、theta(初始化的theta)、alpha（学习率）、num_iters（迭代梯度下降的轮数）
    输出：theta（最终学习得到的theta）、epoch_loss（各轮cost值（有间隔的取样）组成的list,用于绘制cost曲线图）
    """
    m = len(Y)
    epoch_loss = []
    i = 0
    while i < num_iters:
        new_theta = Gradient_Descent(X,Y,theta,alpha)
        theta = new_theta
        # print(theta)
        i = i + 1
        if i % 50 == 0:
            cost = Cost_Function(X, Y, theta)
            epoch_loss.append(cost)
    return theta, epoch_loss 

In [8]:
#超参数设置与初始化
initial_theta = [1,1] #学习参数初始化
alpha = 0.01 #学习率
iterations = 1000 #训练轮数

In [9]:
#opt_theta最终学习到的模型参数theta
opt_theta, epoch_loss = Logistic_Regression(X_train,Y_train,alpha,initial_theta,iterations)
# print(epoch_loss)

# 模型评估与绘制cost曲线 

In [10]:
def cost_plot(epoch_losses, iterations, figure_name, imgPath = "./"):
    plt.switch_backend('Agg') 
    plt.figure() 
    x = list(range(0,iterations,50))
    plt.plot(x, epoch_losses,'b',label = 'loss')
    plt.xlabel('epoch')
    plt.legend('Logistic Regression')        
    plt.savefig(os.path.join(imgPath,figure_name)) 
    
figure_name = "1_recon_loss.jpg"    
cost_plot(epoch_loss,iterations,figure_name)

In [11]:
def evaluation(X_test,Y_test,theta):
    length = len(X_test)
    score = 0
    for i in range(length):
        prediction = round(Hypothesis(X_test[i],theta))#用round函数四舍五入，将概率映射到标签
        answer = Y_test[i]
        if prediction == answer:
            score += 1
    score = float(score) / float(length)
    print('Accuracy = ',score)

evaluation(X_test,Y_test,opt_theta)

Accuracy =  0.8181818181818182


请尝试不同的初始化、学习率、训练轮数来训练模型，并绘制cost曲线，观察准确率